# Measured 190 GHz Active 2-Port

The Vector Fitting feature is demonstrated using a 2-port S-matrix of an active circuit measured from 140 GHz to 220 GHz. Additional explanations and background information can be found in the [Vector Fitting tutorial](../../tutorials/VectorFitting.ipynb).

In [ ]:
import skrf
import numpy as np

This example is a lot more tricky to fit, because the responses contain a few "bumps" and noise from the measurement. In such a case, finding a good number of initial poles can take a few iterations. 

Load the Network from a Touchstone file and create the Vector Fitting instance:

In [ ]:
nw2 = skrf.network.Network('./190ghz_tx_measured.S2P')
vf2 = skrf.VectorFitting(nw2)

**First attempt:** Perform the fit using 4 real poles and 3 complex-conjugate poles with *linear* spacing (default):

In [ ]:
vf2.vector_fit(n_poles_real=4, n_poles_cmplx=3)

The function `plot_convergence()` can be helpful to examine the convergence and see if something was going wrong. In this case, it took quite a while (84 iteration steps), but the results converged nevertheless.

In [ ]:
vf2.plot_convergence()

Checking the results by comparing the model responses to the original sampled data indicates a successful fit:

In [ ]:
vf2.plot_s_mag(0, 0) # s11
vf2.plot_s_mag(1, 0) # s21

It is a good idea to also check the model response well outside the original frequency range. This reveals a strong resonance at higher frequencies (at approx. 330 GHz), which is not ideal:

In [ ]:
freqs2 = np.linspace(0, 500e9, 501)
vf2.plot_s_mag(0, 0, freqs2)
vf2.plot_s_mag(1, 0, freqs2)

**Second attempt:** Maybe an even better fit can be achieved, so let's try again. It sometimes helps to change the initial pole spacing from *linear* to *logarithmic*:

In [ ]:
vf2.vector_fit(n_poles_real=4, n_poles_cmplx=3, init_pole_spacing='log')
vf2.plot_convergence()

This fit converged slightly quicker (66 iteration steps) and also matches the network data very well inside the fitting band. Interestingly, the strong resonance from before in the outside band is replaced with a much weaker one at 263 GHz:

In [ ]:
vf2.plot_s_mag(0, 0)
vf2.plot_s_mag(1, 0)
vf2.plot_s_mag(0, 0, freqs2)
vf2.plot_s_mag(1, 0, freqs2)

This looks good, so let's export the model as a SPICE subcircuit. For example:

`vf2.write_spice_subcircuit_s('/home/vinc/Desktop/190ghz_tx.sp')`

The subcircuit can then be simulated in SPICE with the same AC simulation setup as in the [ring slot example](./vectorfitting_ringslot.ipynb):
<img src="./ngspice_190ghz_tx_sp_mag.svg" />
<img src="./ngspice_190ghz_tx_sp_smith.svg" />